In [1]:
from __future__ import print_function

In [2]:
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from altair import Row, Column, Chart, Text, Scale, Color, X, Y, Bin, SortField
import json

In [29]:
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split

/Users/kbalk/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from library.data_connect import *
from library.plot_tools import *

In [9]:
pd.set_option("display.max_columns",None)

Pull Data

In [43]:
results_df = get_results_old(division=1, stages='all')

In [44]:
results_df.head(3)

,index,athlete_id,division,14.1,14.2,14.3,14.4,14.5,15.1,15.11,15.2,15.3,15.4,15.5,13.1,13.2,13.3,13.4,13.5,name,region,team,affiliate,gender,age,height,weight,fran,helen,grace,filthy50,fgonebad,run400,run5k,candj,snatch,deadlift,backsq,pullups,gender_num
0,0,82,1,169.0,35.0,93.0,160.0,1568.0,NaN,NaN,NaN,NaN,NaN,NaN,100.0,193.0,154.0,36.0,23.0,Adam Reed,Northern California,Games Team 2,Container CrossFit,Male,28.0,73.0,192.0,NaN,NaN,NaN,NaN,0.0,82.0,1440.0,175.0,0.0,280.0,225.0,2.0,1.0
1,1,83,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.0,123.0,NaN,NaN,NaN,Bill McGair,,,,Male,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,2,84,1,308.0,137.0,139.0,187.0,887.0,153.0,205.0,125.0,356.0,97.0,717.0,158.0,300.0,260.0,91.0,109.0,Ryan Johnston,Northern California,LaLanne Fitness CrossFit,LaLanne Fitness CrossFit,Male,25.0,68.0,160.0,182.0,NaN,146.0,1406.0,322.0,NaN,1162.0,215.0,195.0,450.0,330.0,NaN,1.0


Replace NaN

In [47]:

results_df.dropna(axis=0, subset=['15.2'], inplace=True)
results_df.fillna(-99, inplace=True)
results_df.head(3)

,index,athlete_id,division,14.1,14.2,14.3,14.4,14.5,15.1,15.11,15.2,15.3,15.4,15.5,13.1,13.2,13.3,13.4,13.5,name,region,team,affiliate,gender,age,height,weight,fran,helen,grace,filthy50,fgonebad,run400,run5k,candj,snatch,deadlift,backsq,pullups,gender_num
2,2,84,1,308.0,137.0,139.0,187.0,887.0,153.0,205.0,125.0,356.0,97.0,717.0,158.0,300.0,260.0,91.0,109.0,Ryan Johnston,Northern California,LaLanne Fitness CrossFit,LaLanne Fitness CrossFit,Male,25.0,68.0,160.0,182.0,-99.0,146.0,1406.0,322.0,-99.0,1162.0,215.0,195.0,450.0,330.0,-99.0,1.0
3,3,86,1,306.0,132.0,111.0,190.0,1051.0,149.0,232.0,117.0,342.0,46.0,518.0,156.0,271.0,255.0,87.0,76.0,Justin Bergh,Northern California,,Container CrossFit,Male,32.0,77.0,225.0,226.0,496.0,174.0,1287.0,393.0,64.0,1325.0,265.0,210.0,415.0,335.0,32.0,1.0
4,4,88,1,368.0,262.0,149.0,197.0,694.0,188.0,306.0,267.0,491.0,117.0,408.0,164.0,323.0,261.0,105.0,132.0,Cary Hair,Northern California,CrossFit Santa Cruz,CrossFit Santa Cruz,Male,30.0,72.0,200.0,140.0,448.0,109.0,1199.0,407.0,54.0,1294.0,315.0,255.0,518.0,455.0,54.0,1.0


Split Data

In [48]:
y = results_df['15.2'].copy()
y.head(3)

2    125.0
3    117.0
4    267.0
Name: 15.2, dtype: float64

In [55]:
X = results_df.copy().drop(['index', 'name', 'region', 'team', 'affiliate', 'gender', 'gender_num', 'division', '15.2'],1)
X.head(3)

,athlete_id,14.1,14.2,14.3,14.4,14.5,15.1,15.11,15.3,15.4,15.5,13.1,13.2,13.3,13.4,13.5,age,height,weight,fran,helen,grace,filthy50,fgonebad,run400,run5k,candj,snatch,deadlift,backsq,pullups
2,84,308.0,137.0,139.0,187.0,887.0,153.0,205.0,356.0,97.0,717.0,158.0,300.0,260.0,91.0,109.0,25.0,68.0,160.0,182.0,-99.0,146.0,1406.0,322.0,-99.0,1162.0,215.0,195.0,450.0,330.0,-99.0
3,86,306.0,132.0,111.0,190.0,1051.0,149.0,232.0,342.0,46.0,518.0,156.0,271.0,255.0,87.0,76.0,32.0,77.0,225.0,226.0,496.0,174.0,1287.0,393.0,64.0,1325.0,265.0,210.0,415.0,335.0,32.0
4,88,368.0,262.0,149.0,197.0,694.0,188.0,306.0,491.0,117.0,408.0,164.0,323.0,261.0,105.0,132.0,30.0,72.0,200.0,140.0,448.0,109.0,1199.0,407.0,54.0,1294.0,315.0,255.0,518.0,455.0,54.0


In [56]:
train1, test1, target1, testtarget = train_test_split(
    X, y, test_size=0.2, random_state=3)

Initiate Models

In [57]:
rf = RandomForestRegressor(verbose=True)
rf.fit(train1,target1, )

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   11.9s finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=True, warm_start=False)

In [58]:
lr = linear_model.LinearRegression()
lr.fit(train1,target1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Check Models

In [70]:
rf_preddict = rf.predict(test1.head())
rf_preddict

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


array([  64.4,   66.5,  132.7,  122. ,   55.4])

In [71]:
testtarget.head()

44396      83.0
129660     68.0
168047     51.0
43830     114.0
191808     78.0
Name: 15.2, dtype: float64

In [72]:
test1.head()

,athlete_id,14.1,14.2,14.3,14.4,14.5,15.1,15.11,15.3,15.4,15.5,13.1,13.2,13.3,13.4,13.5,age,height,weight,fran,helen,grace,filthy50,fgonebad,run400,run5k,candj,snatch,deadlift,backsq,pullups
44396,169140,307.0,77.0,113.0,190.0,996.0,-99.0,-99.0,339.0,31.0,687.0,100.0,259.0,244.0,61.0,57.0,32.0,72.0,179.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,194.0,-99.0,364.0,254.0,-99.0
129660,107768,-99.0,-99.0,-99.0,-99.0,-99.0,115.0,198.0,540.0,16.0,858.0,100.0,182.0,240.0,42.0,41.0,37.0,72.0,209.0,409.0,612.0,-99.0,-99.0,-99.0,-99.0,1150.0,220.0,172.0,463.0,309.0,-99.0
168047,518831,-99.0,-99.0,-99.0,-99.0,-99.0,195.0,165.0,144.0,83.0,600.0,-99.0,-99.0,-99.0,-99.0,-99.0,31.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
43830,166964,-99.0,81.0,131.0,182.0,984.0,150.0,230.0,332.0,45.0,579.0,146.0,206.0,240.0,68.0,55.0,28.0,72.0,205.0,372.0,-99.0,253.0,-99.0,-99.0,-99.0,1380.0,215.0,155.0,435.0,315.0,-99.0
191808,576161,-99.0,-99.0,-99.0,-99.0,-99.0,150.0,135.0,826.0,80.0,677.0,-99.0,-99.0,-99.0,-99.0,-99.0,29.0,-99.0,195.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [65]:
lr_predict = lr.predict(train1.head())
lr_predict

array([  80.93179235,   86.14274476,  110.67593856,   90.38599216,
         65.67390091])

In [67]:
(rf_preddict + lr_predict)/2

array([  79.21589617,   84.22137238,  107.63796928,   74.54299608,
         80.33695045])

In [68]:
target1.head()

190761     84.0
194361    121.0
681       111.0
180404     63.0
204128     80.0
Name: 15.2, dtype: float64